In [1]:
import os
import pandas as pd
import torch
import numpy as np
os.environ["TRANSFORMERS_CACHE"]="/mnt/swordfish-pool2/models/transformers_cache"

In [2]:
from mutual_implication_score import MIS
mis = MIS(device='cuda:0')

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("s-nlp/roberta-base-formality-ranker")
roberta_model = AutoModelForSequenceClassification.from_pretrained("s-nlp/roberta-base-formality-ranker")

def get_formality_score(txt, tokenizer, model, class_name="formal"):
    if not txt: return 0
    inputs = tokenizer(txt, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    if class_name == "formal":
        return torch.sigmoid(logits)[0][1].item()
    return torch.sigmoid(logits)[0][0].item()

In [7]:
data_dir = "/home/a.saakyan/projects/st/data/gyafc_w_ICHF_alpaca/formal_to_informal"
alpaca_out = f"{data_dir}/alpaca_fif_noexpl_test_output.json"
alpaca_df = pd.read_json(alpaca_out, lines=False)

alpaca_df["informality_score"] = alpaca_df["pred"].apply(lambda x: 
                            get_formality_score(x.split("Informal: ")[1], 
                            tokenizer, 
                            roberta_model,
                            class_name="informal"))
print("Informality Score", alpaca_df["informality_score"].mean())

self_mis = mis.compute(alpaca_df["input"].apply(lambda x: x.split("Formal: ")[1]).tolist(), 
                    alpaca_df["pred"].apply(lambda x: x.split("Informal: ")[1]).tolist() )
print("SELF-MIS", np.mean(self_mis))  

Informality Score 0.5474961273486213


  0%|          | 0/123 [00:00<?, ?it/s]

SELF-MIS 0.8533673458977374


In [8]:
data_dir = "/home/a.saakyan/projects/st/data/gyafc_w_ICHF_alpaca/informal_to_formal"
alpaca_out = f"{data_dir}/alpaca_noexpl_test_output.json"
alpaca_df = pd.read_json(alpaca_out, lines=False)

alpaca_df["formality_score"] = alpaca_df["pred"].apply(lambda x: 
                            get_formality_score(x.split("Formal: ")[1], 
                            tokenizer, 
                            roberta_model))
print("Formality Score", alpaca_df["formality_score"].mean())

self_mis = mis.compute(alpaca_df["input"].apply(lambda x: x.split("Informal: ")[1]).tolist(), 
                    alpaca_df["pred"].apply(lambda x: x.split("Formal: ")[1]).tolist() )
print("SELF-MIS", np.mean(self_mis))  

Formality Score 0.9110360386344243


  0%|          | 0/123 [00:00<?, ?it/s]

SELF-MIS 0.8320098246390722
